<a href="https://colab.research.google.com/github/JimohAR/it_core_project1/blob/main/notebooks/task1_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [2]:
if "google.colab" in str(get_ipython()):
    !pip install dvc[gdrive] > /content/piplog.txt
    print(">> g-colab detected \ncloning repo from github\n")
    !git clone https://github.com/JimohAR/it_core_project1

    print("\n>> changing path to the repo\n")
    os.chdir("/content/it_core_project1")

    print("\n>> downloading the datasets\n")
    !dvc import https://github.com/JimohAR/it_core_project1 data/telco.csv \
    -o data/telco.csv

    print("\n>> set up path to the data directory")
    path = os.getcwd() + "/data/"
else:
    print("\n>> set up path to the data directory")
    path = os.path.abspath(os.getcwd() + "/../../data") + "/"


>> set up path to the data directory


In [3]:
data = pd.read_csv(path + "telco.csv").iloc[:-1]

In [4]:
desc = pd.read_excel(path + "field_descriptions.xlsx").set_index("Fields")

In [5]:
data.select_dtypes(exclude ="object").describe()

,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
count,1.490100e+05,150000.000000,150000.000000,1.500000e+05,1.494310e+05,1.489350e+05,1.494290e+05,122172.000000,122189.000000,150000.000000,...,1.500000e+05,1.500000e+05,1.500000e+05,1.500000e+05,1.500000e+05,1.500000e+05,1.500000e+05,1.500000e+05,1.500000e+05,1.500000e+05
mean,1.013887e+19,499.188200,498.800880,1.046086e+05,2.082016e+14,4.188282e+10,4.847455e+13,109.795706,17.662883,13300.045927,...,1.163407e+07,1.100941e+07,1.162685e+07,1.100175e+07,4.220447e+08,8.288398e+06,4.211005e+08,8.264799e+06,4.112121e+07,4.546434e+08
std,2.893173e+18,288.611834,288.097653,8.103762e+04,2.148809e+10,2.447443e+12,2.241637e+13,619.782739,84.793524,23971.878541,...,6.710591e+06,6.345445e+06,6.725240e+06,6.359511e+06,2.439683e+08,4.782716e+06,2.432058e+08,4.769020e+06,1.127639e+07,2.441429e+08
min,6.917538e+18,0.000000,0.000000,7.142000e+03,2.040471e+14,3.360100e+10,4.400152e+11,0.000000,0.000000,0.000000,...,5.300000e+01,1.050000e+02,4.200000e+01,3.500000e+01,2.516000e+03,5.900000e+01,3.290000e+03,1.480000e+02,2.866892e+06,7.114041e+06
25%,7.349883e+18,250.000000,251.000000,5.744050e+04,2.082014e+14,3.365130e+10,3.546071e+13,32.000000,2.000000,43.000000,...,5.833482e+06,5.517962e+06,5.777095e+06,5.475977e+06,2.104706e+08,4.128424e+06,2.101824e+08,4.145904e+06,3.322201e+07,2.431068e+08
50%,7.349883e+18,499.000000,500.000000,8.639900e+04,2.082015e+14,3.366371e+10,3.572201e+13,45.000000,5.000000,63.000000,...,1.161601e+07,1.101348e+07,1.164222e+07,1.099631e+07,4.234082e+08,8.291215e+06,4.218057e+08,8.267086e+06,4.114331e+07,4.558411e+08
75%,1.304243e+19,749.000000,750.000000,1.324302e+05,2.082018e+14,3.368349e+10,8.611970e+13,70.000000,15.000000,19710.750000,...,1.744855e+07,1.651559e+07,1.747050e+07,1.650729e+07,6.331747e+08,1.243163e+07,6.316950e+08,1.238420e+07,4.903424e+07,6.657055e+08
max,1.318654e+19,999.000000,999.000000,1.859336e+06,2.140743e+14,8.823971e+14,9.900120e+13,96923.000000,7120.000000,378160.000000,...,2.325910e+07,2.201196e+07,2.325919e+07,2.201196e+07,8.434419e+08,1.655879e+07,8.434425e+08,1.655882e+07,7.833131e+07,9.029696e+08


## User Overview Analysis
* Start by identifying the top 10 handsets used by the customers.

* Then, identify the top 3 handset manufacturers

* Next, identify the top 5 handsets per top 3 handset manufacturer

* Make a short interpretation and recommendation to marketing teams

In [6]:
new_data_col = [i.strip() for i in list(data.columns)]

In [7]:
data.columns = new_data_col

In [8]:
handset_data = data[['Handset Manufacturer', 'Handset Type',]]

In [9]:
top10_handet = handset_data["Handset Type"][handset_data["Handset Type"] != "undefined"].value_counts()[:10]#.plot(kind = "bar")

In [10]:
top3_manufacturers = handset_data["Handset Manufacturer"][handset_data["Handset Manufacturer"] != "undefined"].value_counts().head(3)

In [11]:
handsets_by_top3_manufacturers = handset_data.set_index(keys= ["Handset Manufacturer"]).loc[top3_manufacturers.keys()]

In [12]:
top5_of_top3_manufacturers_dict = dict()
for i in top3_manufacturers.keys():
    top5_of_top3_manufacturers_dict[i] = handsets_by_top3_manufacturers.loc[i, "Handset Type"].value_counts().head(5)

In [13]:
top5_of_top3_manufacturers = pd.DataFrame.from_dict(top5_of_top3_manufacturers_dict, orient= "index").stack()
top5_of_top3_manufacturers.name = "count"
top5_of_top3_manufacturers = top5_of_top3_manufacturers.to_frame()

## Task 1.1
Aggregate per user the following information in the column  
- number of xDR sessions
- session duration
- the total download (DL) and upload (UL) data
- the total data volume (in Bytes) during this session for each application

In [14]:
data["MSISDN/Number"] = data["MSISDN/Number"].fillna(-999).astype("int64")

In [15]:
no_of_xdr_sessions = data["MSISDN/Number"].fillna("median").value_counts()

In [16]:
total_sessions_duration = (data[["MSISDN/Number", "Dur. (ms)"]]
                           .fillna("median").groupby(["MSISDN/Number"])["Dur. (ms)"]
                           .sum()
                          )/(60*60)

In [17]:
total_dl_ul_data = (data[["MSISDN/Number", "Total DL (Bytes)", "Total UL (Bytes)"]]
                    .fillna("median").groupby(["MSISDN/Number"])[["Total DL (Bytes)", "Total UL (Bytes)"]]
                    .sum()
                   )/(2**20)

In [18]:
total_dl_ul_data["total_data"] = total_dl_ul_data.sum(1)

In [19]:
apps = ['Social Media DL (Bytes)', 'Social Media UL (Bytes)',
       'Google DL (Bytes)', 'Google UL (Bytes)', 'Email DL (Bytes)',
       'Email UL (Bytes)', 'Youtube DL (Bytes)', 'Youtube UL (Bytes)',
       'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)',
       'Gaming UL (Bytes)', 'Other DL (Bytes)', 'Other UL (Bytes)',]

In [20]:
total_dl_ul_data_per_app = (data[apps + ["MSISDN/Number"]].fillna("mean")
                            .groupby(["MSISDN/Number"])[apps].sum()
                           )/(2**20)

In [21]:
total_data_per_app = pd.DataFrame()
count = 0
for i,j in enumerate(total_dl_ul_data_per_app.columns):
    if count != 1:
        total_data_per_app[j.split()[0]] = total_dl_ul_data_per_app.iloc[:, i] + total_dl_ul_data_per_app.iloc[:,i+1]
        count = (count + 1) % 2
    else: 
        count = (count + 1) % 2
        continue

In [22]:
aggregates = (no_of_xdr_sessions.to_frame()
              .join(total_sessions_duration)
              .join(total_data_per_app)
              .join(total_dl_ul_data)
             )

In [23]:
aggregates.index.name = "MSISDN"
aggregates.columns = ["tot number of session", "tot sessions duration (hrs)", "social media data usage (MBs)",
                      "google data usage (MBs)","Email data usage (MBs)", "youtube data usage (MBs)", 
                      "netflix data usage (MBs)",  "gaming apps data usage (MBs)", "other apps data usage (MBs)", 
                      "tot downloaded data (MBs)", "tot uploaded data (MBs)", "tot data usage (MBs)",]
aggregates = aggregates.drop(index = -999)

In [24]:
aggregates.describe().style#.set_properties(**{"width": "100px"})

,tot number of session,tot sessions duration (hrs),social media data usage (MBs),google data usage (MBs),Email data usage (MBs),youtube data usage (MBs),netflix data usage (MBs),gaming apps data usage (MBs),other apps data usage (MBs),tot downloaded data (MBs),tot uploaded data (MBs),tot data usage (MBs)
count,106856.000000,106856.000000,106856.000000,106856.000000,106856.000000,106856.000000,106856.000000,106856.000000,106856.000000,106856.000000,106856.000000,106856.000000
mean,1.393792,40.601988,2.429931,10.378298,3.002926,30.096435,30.077298,571.984301,570.782437,604.298287,54.654524,658.952812
std,0.806022,51.766297,1.819647,7.195341,2.119446,20.308416,20.303304,428.343260,428.125486,443.034226,34.005785,468.307391
min,1.000000,1.983889,0.001491,0.038462,0.017334,0.075248,0.176019,0.292166,0.142140,8.418161,2.734081,31.708726
25%,1.000000,19.807778,1.155168,5.667339,1.596910,17.767991,17.696355,274.718392,272.331153,300.242485,34.709430,341.939838
50%,1.000000,28.538889,2.197033,9.142068,2.670121,25.558830,25.481119,517.224509,514.824241,543.945048,44.626108,589.297426
75%,2.000000,47.999722,3.154286,12.602558,3.668973,36.170936,36.217649,741.294992,739.476710,769.962778,62.686913,817.713878
max,18.000000,5153.820556,41.365413,145.141460,40.453703,431.975144,381.011085,7390.434488,7789.495255,7778.876775,695.779209,8436.419004


### removing outliers : any user with session duration greater than (30*12) mins 

In [25]:
outliers_index = \
aggregates["tot sessions duration (hrs)"][aggregates["tot sessions duration (hrs)"] > (30*12)].keys()
aggregates.loc[outliers_index] = np.nan
aggregates.dropna(inplace=True)

In [26]:
aggregates.describe().style

,tot number of session,tot sessions duration (hrs),social media data usage (MBs),google data usage (MBs),Email data usage (MBs),youtube data usage (MBs),netflix data usage (MBs),gaming apps data usage (MBs),other apps data usage (MBs),tot downloaded data (MBs),tot uploaded data (MBs),tot data usage (MBs)
count,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000
mean,1.379986,38.945798,2.406328,10.273692,2.972841,29.795385,29.777472,566.283479,565.158929,598.270156,54.113350,652.383505
std,0.751298,34.344976,1.744367,6.838355,2.016566,19.260624,19.266514,409.946436,410.941550,423.254635,31.994931,446.221514
min,1.000000,1.983889,0.001491,0.038462,0.017334,0.075248,0.176019,0.292166,0.142140,8.418161,2.734081,31.708726
25%,1.000000,19.710625,1.152503,5.656567,1.593564,17.742988,17.674101,274.037992,271.572208,299.494924,34.676584,341.003925
50%,1.000000,28.456111,2.191891,9.124181,2.664310,25.511245,25.433387,515.971701,513.621295,542.765436,44.576648,587.853882
75%,2.000000,47.999722,3.146709,12.565546,3.659199,36.047137,36.091989,739.504440,737.657516,767.978338,62.295820,815.835433
max,11.000000,359.504444,22.698244,83.574781,21.861470,237.480580,222.896995,4343.421737,4392.022408,4545.041459,435.936921,4860.047088


## Task 1.2

○	Describe all relevant variables and associated data types (slide). 

○	Analyze the basic metrics (mean, median, etc) in the Dataset (explain) & their importance for the global objective.

○	Conduct a Non-Graphical Univariate Analysis by computing dispersion parameters for each quantitative variable and providing useful interpretation. 

○	Conduct a Graphical Univariate Analysis by identifying the most suitable plotting options for each variable and interpreting your findings.

○	Bivariate Analysis – explore the relationship between each application & the total DL+UL data using appropriate methods and interpret your findings. 

○	Variable transformations – segment the users into the top five decile classes based on the total duration for all sessions and compute the total data (DL+UL) per decile class. 

○	Correlation Analysis – compute a correlation matrix for the following variables and interpret your findings: Social Media data, Google data, Email data, Youtube data, Netflix data, Gaming data, Other data 

○	Dimensionality Reduction – perform a principal component analysis to reduce the dimensions of your data and provide a useful interpretation of the results (Provide your interpretation in four (4) bullet points-maximum). 


In [27]:
aggregates#.dtypes

,tot number of session,tot sessions duration (hrs),social media data usage (MBs),google data usage (MBs),Email data usage (MBs),youtube data usage (MBs),netflix data usage (MBs),gaming apps data usage (MBs),other apps data usage (MBs),tot downloaded data (MBs),tot uploaded data (MBs),tot data usage (MBs)
MSISDN,,,,,,,,,,,,
33669068942,11.0,169.666944,22.698244,74.219402,21.861470,225.451283,202.187567,3117.705966,4392.022408,3316.770946,408.798099,3725.569044
33665083783,10.0,240.018333,14.433722,83.574781,20.907565,237.480580,222.896995,3099.030754,4337.693751,3322.831337,435.936921,3758.768258
33650426463,9.0,214.266111,17.218717,75.072834,17.417238,178.700468,146.218878,4343.421737,2909.191833,4545.041459,315.005629,4860.047088
33669710478,8.0,182.404167,13.665357,40.924310,17.124997,226.046673,161.152156,2832.951587,1629.431746,3033.195561,332.063877,3365.259439
33666169635,8.0,166.000000,15.429578,50.629637,16.440080,180.657377,192.058394,4050.096912,2158.593174,4262.374310,288.527947,4550.902257
...,...,...,...,...,...,...,...,...,...,...,...,...
33664955942,1.0,43.766944,0.911378,7.403456,2.962868,21.082132,27.041847,617.488104,767.827761,648.684183,40.332630,689.016813
33663469453,1.0,47.070000,3.161198,5.704038,1.098277,39.902770,19.359421,173.002231,317.961584,199.902887,55.848002,255.750890
33665287654,1.0,49.665278,1.709334,0.549103,1.040939,19.620629,19.510183,487.250133,416.906700,508.455457,31.720077,540.175533


In [28]:
ss = aggregates.describe().loc[["mean", "min", "50%", "75%", "max"]].T

In [29]:
ss.style.set_precision(2).set_properties(**{"width": "100px", "color": "blue"})

C:\Users\abola\AppData\Local\Temp/ipykernel_13288/4215655550.py:1: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  ss.style.set_precision(2).set_properties(**{"width": "100px", "color": "blue"})


,mean,min,50%,75%,max
tot number of session,1.38,1.00,1.00,2.00,11.00
tot sessions duration (hrs),38.95,1.98,28.46,48.00,359.50
social media data usage (MBs),2.41,0.00,2.19,3.15,22.70
google data usage (MBs),10.27,0.04,9.12,12.57,83.57
Email data usage (MBs),2.97,0.02,2.66,3.66,21.86
youtube data usage (MBs),29.80,0.08,25.51,36.05,237.48
netflix data usage (MBs),29.78,0.18,25.43,36.09,222.90
gaming apps data usage (MBs),566.28,0.29,515.97,739.50,4343.42
other apps data usage (MBs),565.16,0.14,513.62,737.66,4392.02
tot downloaded data (MBs),598.27,8.42,542.77,767.98,4545.04


In [30]:
# aggregates.iloc[:,:1].boxplot(figsize= (20,10), whis= (0,100))

In [31]:
# aggregates.iloc[:,1:2].boxplot(figsize= (20,10), whis= (0,100))

In [32]:
# aggregates.iloc[:,2:7].boxplot(figsize= (20,10), whis= (0,100))

In [33]:
# aggregates.iloc[:,7:12].boxplot(figsize= (20,10), whis= (0,100))

In [34]:
sessions_and_data_used_sorted = \
(aggregates[["tot sessions duration (hrs)", "tot data usage (MBs)"]]
 .sort_values(["tot sessions duration (hrs)"], ascending= False)
)

In [35]:
sessions_binned = pd.qcut(sessions_and_data_used_sorted.iloc[:,0], 10, labels=np.arange(10))
sessions_binned.name = "sessions_bin_class"

In [36]:
tot_sessions_and_data_used_binned = (sessions_and_data_used_sorted.join(sessions_binned)
                                     .groupby("sessions_bin_class").aggregate("mean")
                                    )

In [37]:
tot_sessions_and_data_used_binned

,tot sessions duration (hrs),tot data usage (MBs)
sessions_bin_class,,
0,5.567984,485.476552
1,11.143625,544.368834
2,19.699618,581.556603
3,23.990098,476.309916
4,26.147770,531.532518
5,32.491201,573.819973
6,40.890934,548.189196
7,48.434390,694.320787
8,64.085545,827.252899


In [38]:
aggregates.describe()

,tot number of session,tot sessions duration (hrs),social media data usage (MBs),google data usage (MBs),Email data usage (MBs),youtube data usage (MBs),netflix data usage (MBs),gaming apps data usage (MBs),other apps data usage (MBs),tot downloaded data (MBs),tot uploaded data (MBs),tot data usage (MBs)
count,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000
mean,1.379986,38.945798,2.406328,10.273692,2.972841,29.795385,29.777472,566.283479,565.158929,598.270156,54.113350,652.383505
std,0.751298,34.344976,1.744367,6.838355,2.016566,19.260624,19.266514,409.946436,410.941550,423.254635,31.994931,446.221514
min,1.000000,1.983889,0.001491,0.038462,0.017334,0.075248,0.176019,0.292166,0.142140,8.418161,2.734081,31.708726
25%,1.000000,19.710625,1.152503,5.656567,1.593564,17.742988,17.674101,274.037992,271.572208,299.494924,34.676584,341.003925
50%,1.000000,28.456111,2.191891,9.124181,2.664310,25.511245,25.433387,515.971701,513.621295,542.765436,44.576648,587.853882
75%,2.000000,47.999722,3.146709,12.565546,3.659199,36.047137,36.091989,739.504440,737.657516,767.978338,62.295820,815.835433
max,11.000000,359.504444,22.698244,83.574781,21.861470,237.480580,222.896995,4343.421737,4392.022408,4545.041459,435.936921,4860.047088


In [39]:
corr_data = aggregates[["social media data usage (MBs)", "google data usage (MBs)", "Email data usage (MBs)", "youtube data usage (MBs)",
"netflix data usage (MBs)", "gaming apps data usage (MBs)", "other apps data usage (MBs)"]].corr()

In [40]:
corr_data

,social media data usage (MBs),google data usage (MBs),Email data usage (MBs),youtube data usage (MBs),netflix data usage (MBs),gaming apps data usage (MBs),other apps data usage (MBs)
social media data usage (MBs),1.000000,0.612185,0.603240,0.630696,0.631429,0.558067,0.560730
google data usage (MBs),0.612185,1.000000,0.658705,0.690565,0.688525,0.610693,0.611985
Email data usage (MBs),0.603240,0.658705,1.000000,0.676026,0.676933,0.595747,0.600171
youtube data usage (MBs),0.630696,0.690565,0.676026,1.000000,0.711979,0.626970,0.630931
netflix data usage (MBs),0.631429,0.688525,0.676933,0.711979,1.000000,0.627023,0.625519
gaming apps data usage (MBs),0.558067,0.610693,0.595747,0.626970,0.627023,1.000000,0.554267
other apps data usage (MBs),0.560730,0.611985,0.600171,0.630931,0.625519,0.554267,1.000000


#### performing PCA on aggregates

In [41]:
## normalize data
scaler = MinMaxScaler()
agg_scaled = scaler.fit_transform(aggregates)

In [42]:
pd.DataFrame(PCA(1, random_state= 0).fit_transform(agg_scaled)).describe().style

,0
count,106554.000000
mean,0.000000
std,0.250138
min,-0.326989
25%,-0.148936
50%,-0.087153
75%,0.079829
max,2.567021


In [43]:
pd.DataFrame(agg_scaled).describe().style

,0,1,2,3,4,5,6,7,8,9,10,11
count,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000,106554.000000
mean,0.037999,0.103384,0.105955,0.122524,0.135300,0.125187,0.132908,0.130319,0.128650,0.130020,0.118603,0.128548
std,0.075130,0.096064,0.076855,0.081861,0.092316,0.081130,0.086505,0.094390,0.093568,0.093297,0.073857,0.092417
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.049582,0.050713,0.067253,0.072158,0.074420,0.078565,0.063030,0.061803,0.064162,0.073736,0.064058
50%,0.000000,0.074044,0.096507,0.108764,0.121176,0.107142,0.113404,0.118735,0.116916,0.117785,0.096589,0.115184
75%,0.100000,0.128708,0.138576,0.149960,0.166720,0.151521,0.161260,0.170203,0.167927,0.167429,0.137492,0.162401
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [44]:
aggregates.to_csv(path + "aggregates.csv")